**Calculating P-Alive, CLTV and predicted purchase value**

In [0]:
# Importing necessary libraries
from pyspark.sql.types import *
from pyspark.sql.functions import pandas_udf, PandasUDFType
from lifetimes import ModifiedBetaGeoFitter
from lifetimes import GammaGammaFitter
import warnings
warnings.filterwarnings("ignore")
from pyspark.sql import SparkSession   # pyspark session
spark = SparkSession.builder.appName("Hive test").enableHiveSupport().getOrCreate()  #create a spark object and enable hive
sqlContext.setConf("spark.sql.parquet.writeLegacyFormat", "true") # To enable all parquet files with same datatypes 

# Using Carrefour_v1 database 
spark.sql('use cpr_officedepot')

Out[1]: DataFrame[]

In [0]:
# Deriving R,F,M and T columns
frequency_monetary_data = spark.sql("""
select cast(customer_id as int), freq - 1 as frequency, case when gross_sales <= 0 then 0.001 else gross_sales / (freq) end as monetary_value 
from 
(select distinct customer_id ,count(distinct transaction_date) as freq, sum(gross_sales) as gross_sales 
from gm_product_profit_fact 
group by customer_id 
order by customer_id)""")
recency_T_data = spark.sql("""
select cast(customer_id as int) , (T-recency) as recency , T 
from (select distinct customer_id , datediff(max(transaction_date) over(),max(transaction_date) over(partition by customer_id)) as recency, datediff(max(transaction_date) over(),min(transaction_date) over(partition by customer_id)) as T 
from gm_product_profit_fact 
order by customer_id)""")

frequency_monetary_data.join(recency_T_data,on='customer_id',how='inner').createOrReplaceTempView("cltv")

In [0]:
# Creating 8 partitions to process the data in batch wise
clv=spark.sql("""
select (row_number() over (order by customer_id)) % 8 as partition_id, * 
from cltv order by customer_id""").repartition(sc.defaultParallelism, ['partition_id'])

In [0]:
# Creating Schema
result_schema =StructType([
  StructField('customer_id',IntegerType()),
  StructField('palive_value',DoubleType()),
  StructField('predicted_purchases_30days',DoubleType()),
  StructField('predicted_purchases_60days',DoubleType()),
  StructField('predicted_purchases_90days',DoubleType()),
  StructField('predicted_purchases_120days',DoubleType()),
  StructField('predicted_purchases_270days',DoubleType()),
  StructField('predicted_purchases_360days',DoubleType()),
  StructField('customer_value_6months',DoubleType()),
  StructField('customer_value_12months',DoubleType()),
  StructField('customer_value_18months',DoubleType()),
  StructField('customer_value_24months',DoubleType()),
  ])

@pandas_udf(result_schema, PandasUDFType.GROUPED_MAP )
def apply_prediction(data):
    data = data.set_index('customer_id')
    data[data['frequency'] <= 0] = 1

    #calculating Palive and predicted purchases as well as CLTV

    #fitting BetaGeoFitter ( NBD model)
    bgf = ModifiedBetaGeoFitter(penalizer_coef=0.1) #penalizers on the order of 0.001 to 0.1 are effective
    bgf.fit(data['frequency'], data['recency'], data['T'])
    
    #calculating palive score
    data['palive_value'] = bgf.conditional_probability_alive(data['frequency'], data['recency'], data['T'])

    t = 30
    data['predicted_purchases_30days'] = bgf.conditional_expected_number_of_purchases_up_to_time(t, data['frequency'], data['recency'], data['T'])
    data['predicted_purchases_60days'] = bgf.conditional_expected_number_of_purchases_up_to_time(60, data['frequency'], data['recency'], data['T'])
    data['predicted_purchases_90days'] = bgf.conditional_expected_number_of_purchases_up_to_time(90, data['frequency'], data['recency'], data['T'])

    data['predicted_purchases_120days'] = bgf.conditional_expected_number_of_purchases_up_to_time(120, data['frequency'], data['recency'], data['T'])
    data['predicted_purchases_270days'] = bgf.conditional_expected_number_of_purchases_up_to_time(270, data['frequency'], data['recency'], data['T'])
    data['predicted_purchases_360days'] = bgf.conditional_expected_number_of_purchases_up_to_time(360, data['frequency'], data['recency'], data['T'])

   
    #fitting gamma model
    ggf = GammaGammaFitter(penalizer_coef = 0.1)
    ggf.fit(data['frequency'],
            data['monetary_value'])

    
    #calculative cltv for next months
    data['customer_value_6months'] = ggf.customer_lifetime_value(
        bgf, #the model to use to predict the number of future transactions
        data['frequency'],
        data['recency'],
        data['T'],
        data['monetary_value'],
        time=6, # months
        discount_rate=0.01 # monthly discount rate ~ 12.7% annually -- consider discounted cash flow
    )

    data['customer_value_12months'] = ggf.customer_lifetime_value(
        bgf, #the model to use to predict the number of future transactions
        data['frequency'],
        data['recency'],
        data['T'],
        data['monetary_value'],
        time=12, # months
        discount_rate=0.01 # monthly discount rate ~ 12.7% annually -- consider discounted cash flow
    )

    data['customer_value_18months'] = ggf.customer_lifetime_value(
        bgf, #the model to use to predict the number of future transactions
        data['frequency'],
        data['recency'],
        data['T'],
        data['monetary_value'],
        time=18, # months
        discount_rate=0.01 # monthly discount rate ~ 12.7% annually -- consider discounted cash flow
    )

    data['customer_value_24months'] = ggf.customer_lifetime_value(
        bgf, #the model to use to predict the number of future transactions
        data['frequency'],
        data['recency'],
        data['T'],
        data['monetary_value'],
        time=24, # months
        discount_rate=0.01 # monthly discount rate ~ 12.7% annually -- consider discounted cash flow
    )
    data = data.reset_index()
  
    return data[ ['customer_id', 'palive_value', 'predicted_purchases_30days', 'predicted_purchases_60days', 'predicted_purchases_90days', 'predicted_purchases_120days', 'predicted_purchases_270days','predicted_purchases_360days','customer_value_6months','customer_value_12months','customer_value_18months','customer_value_24months'] ]  

In [0]:
results = (clv.groupBy('partition_id').apply(apply_prediction))

In [0]:
results.write.mode("overwrite").option("path","/mnt/mountName/hive/warehouse/cpr_officedepot.db/customer_palive_clv_fullperiod").saveAsTable("cpr_officedepot.customer_palive_clv_fullperiod")

In [0]:
%sql
select * from cpr_officedepot.customer_palive_clv_fullperiod where customer_id = '861285630'

customer_id palive_value predicted_purchases_30days predicted_purchases_60days predicted_purchases_90days predicted_purchases_120days predicted_purchases_270days predicted_purchases_360days customer_value_6months customer_value_12months customer_value_18months customer_value_24months 861285630 0.9986275910023331 0.3597817202734694 0.7195312932185967 1.079249689626639 1.4389378234650323 3.2369531332170007 4.315447759814634 1170.505815003159 2272.66103776339 3310.524668388683 4287.895092848319